In [1]:
import gym
import folium

import numpy as np
import pandas as pd

from h3 import h3
from gym import spaces
from folium.features import DivIcon
from IPython.display import display
from ride_hailing_match import Match
from ride_hailing_location_model import Build_Model
from pyproj import Transformer


Global_Resolution = 6 # change the resolution here, determines the number of hexagonal cells

C:\Users\zhh\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


## This is the source code to be modified.

## Test cells below

In [28]:
"""
# Ride_hailing env
This is the file for defining the simulator for the ride-hailing environment.
Functions defined in this file can be used for reinforcement learning.
This is part of the master thesis project:
Optimising matching radius for a ride-hailing system.

# Use this .py script:
env = RideHailingENV()
radius = env.reset()
time_step = 1
reward, next_state, matched_ride = env.step(radius, time_step, rend_step=False)

# Test this environment for one step:
env = RideHailingENV()
radius = env.reset()
time_step = 1
reward, matched_ride = env.step(radius, time_step, rend_step=True)
print(reward, matched_ride)

# Test this environment for more steps:
import random
time_step += 1
for i in range(np.size(radius)):
    radius[i] = random.randint(50, 3000)
reward, matched_ride = env.step(radius, time_step, rend_step=True)
print(reward, matched_ride)

# Test this environment for one episode (12 hours):
import random
radius = env.reset()
time_step = 1
import random
for i in range(30*16):
    time_step += 0
    for i in range(np.size(radius)):
        radius[i] = random.randint(50, 3000)
    reward, matched_ride = env.step(radius, time_step, rend_step=False)
print(reward, matched_ride)
"""


import gym
import folium

import numpy as np
import pandas as pd

from h3 import h3
from gym import spaces
from folium.features import DivIcon
from IPython.display import display
from ride_hailing_match import Match
from ride_hailing_location_model import Build_Model
from pyproj import Transformer


Global_Resolution = 6 # change the resolution here, determines the number of hexagonal cells


def wgs84_to_xy(x_arr: np.ndarray, y_arr: np.ndarray):
    transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32614')
    x0 = 604082.94
    y0 = 3328141.76
    x_arr_new, y_arr_new = transformer.transform(x_arr, y_arr)
    x_arr_new -= x0
    y_arr_new -= y0
    return x_arr_new.tolist(), y_arr_new.tolist()

def xy_to_wgs84(xy_list):
    transformer = Transformer.from_crs('EPSG:32614', 'EPSG:4326')
    x_new = np.array(xy_list[0]) + 604082.94
    y_new = np.array(xy_list[1]) + 3328141.76
    lat_lon = transformer.transform(x_new, y_new)
    return lat_lon

def xy_to_wgs84_list(xy_list):
    transformer = Transformer.from_crs('EPSG:32614', 'EPSG:4326')
    x_arr = np.array(xy_list[0]) + 604082.94
    y_arr = np.array(xy_list[1]) + 3328141.76
    lat_list, lon_list = transformer.transform(x_arr, y_arr)
    return lat_list.tolist(), lon_list.tolist()


class HexH3:
    """Gennerate H3 hexagonal cells

    This class is to gennerate hexagonal cells base on H3 package developed by Uber team.
    The genenrated H3 cells are provided with a H3 code of each cell. In H3 defination, 
    each different cell has its unique H3 code, more information can be found on H3 official
    website. This class is set default to gennerate H3 cells the urban area of Austin, Texas, USA.

    Attributes:
     lat_range - a tuple of latitude range, defines the location where H3 cells are gennerated
     lon_range - a tuple of longitude range, defines the location where H3 cells are gennerated
    """

    def __init__(self) -> None:
        self.lat_range = [30.10, 30.54] # Austin latitude range
        self.lon_range = [-97.96, -97.54] # Austin longitude range
        pass
   
    def get_hexagons(self, resolution: int = 6, display_map: bool = False) -> list:
        """Gennerate H3 codes for studied area

        Gennerate hexagonal codes and their H3 codes for the studied area,
        with the package of H3 developed by Uber.

        Parameters:
         resolution - an int, with each the hexagonal cells will be generated.
            This dedermines the number of the cells, the smaller this values is,
            the more cells will be gennerated.
         display - a bool, determines display the cells on the map or not.

        Returns:
         hexagons - a list, containing gennerated H3 codes for studied area.
         m - a folium map, only return if argument display is set as True. This is a
            map of studied area with gennerated cells and boundary.
        """
        geoJson = {'type': 'Polygon',
        'coordinates': [[[self.lat_range[0], self.lon_range[0]],
                        [self.lat_range[0], self.lon_range[1]],
                        [self.lat_range[1], self.lon_range[1]],
                        [self.lat_range[1], self.lon_range[0]],
                        ]] }

        hexagons = np.array(list(h3.polyfill(geoJson, resolution)))

        if not display_map:
             return hexagons
        else:
            polyline = geoJson['coordinates'][0]
            polyline.append(polyline[0])
            lat = [p[0] for p in polyline]
            lng = [p[1] for p in polyline]
            m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=12, tiles='cartodbpositron')
            my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color="green")
            m.add_child(my_PolyLine)
            polylines = []
            lat = []
            lng = []
            hex_id = 0
            for hex in hexagons:
                polygons = h3.h3_set_to_multi_polygon([str(hex)], geo_json=False)
                # flatten polygons into loops.
                outlines = [loop for polygon in polygons for loop in polygon]
                polyline = [outline + [outline[0]] for outline in outlines][0]
                lat_text = [p[0] for p in polyline]
                lng_text = [p[1] for p in polyline]
                folium.map.Marker(
                    [sum(lat_text)/len(lat_text), sum(lng_text)/len(lng_text)-0.01],
                    icon=DivIcon(
                        icon_size=(250,36),
                        icon_anchor=(0,0),
                        html=f'<div style="font-size: 20pt">{hex} hex_id:{hex_id}</div>',
                        ) # print hex h3 code and corresponding hex id on the map
                    ).add_to(m)
                lat.extend(map(lambda v:v[0],polyline))
                lng.extend(map(lambda v:v[1],polyline))
                polylines.append(polyline)
                hex_id +=1
            for polyline in polylines:
                my_PolyLine=folium.PolyLine(locations=polyline,weight=1,color='blue')
                m.add_child(my_PolyLine)
            display(m)

            return hexagons
        
    def hex_h3_to_geo(self, hex_h3):
        """
        Transform hex h3 code to geographical coordinates of six edge nodes.

        Parameters:
            hex_h3: a list of h3 codes, codes' type must be string.

        Returns:
            A pandas DataFrame including geographical information for hexagonal cells.
        """
        hex_geo = pd.DataFrame(columns=['hex_id', 'hex_h3', 'east', 'north_east', 'north_west', 'south_east', 'south_west', 'west'])
        for hex_id, hex_code in enumerate(hex_h3):
            geo_info = list(h3.h3_to_geo_boundary(str(hex_code), geo_json=False))
            hex_geo.loc[hex_id] = [hex_id, hex_code] + geo_info
        return hex_geo
        

class Gen_Model:
    """Sample locations from fitted model for riders and drivers in the map

    This class is to generate locations for riders and drivers based on the given
    distribution of their locations. This default model is estimated with Kernel 
    Density Estimation (KDE). The generated location is given in the format of 
    a pandas DataFrame, each row represents a unique rider/driver. Information
    given in a row includes rider/driver's ID, H3 code, longitude and latitude.

    Attributes:
        hexagons: a list of H3 codes of hexagon cells, indicating the studied area.
        hexagons_dic: a dictionary, linking the local ID to its H3 code for H3 cells
        model: an instance of Build_Model class containing models for riders and drivers
        rider_model: a dictionary for 24 KDE distributions, describing the locational and timely distribution of riders
        driver_model: a dictionary for 24 KDE distributions, describing the locational and timely distribution of drivers
    """
    def __init__(self, resolution: int = Global_Resolution, random_seed = 1) -> None:
        self.hex_h3 = HexH3()
        self.hexagons = self.hex_h3.get_hexagons(resolution=resolution, display_map=False)
        self.hexagons_dic = {h3_code: index for index, h3_code in enumerate(self.hexagons)}
        self.model = Build_Model()
        self.rider_model, self.driver_model = self.model.get_model()
        self.random_seed = random_seed

    def gen_drivers(self, number_of_drivers: int, hr_time: int, resolution: int = Global_Resolution):
        """Sample locations for drivers
        
        Sample multiple locations for drivers based on the given locational distribution 
        of drivers. The default distribution model is KDE.

        Parameters:
            number_of_drivers: an int, indicating how many drivers are generated.
            hr_time: an int, the value is the hour of the day, indicating which distribution model will be used.
            resolution: an int, determines the number of cells in the map.

        Returns:
            driver_df: a pandas DataFrame, including information of generated drivers. 
        """
        driver_locations = self.model.sample_from_model(self.driver_model[f'{hr_time}'], number_of_drivers, self.random_seed) # dtype = numpy ndarray
        driver_ids = []
        hex_ids = []

        for driver_id, geo_info in enumerate(driver_locations):
            hex_ids.append(self.hexagons_dic[h3.geo_to_h3(geo_info[0], geo_info[1], resolution)])
            driver_ids.append(driver_id)

        x_list, y_list = wgs84_to_xy(driver_locations.T[0], driver_locations.T[1])

        driver_df = pd.DataFrame({'driver_id':driver_ids, 'hex_id':hex_ids, 'x':x_list, 'y':y_list})
        # driver_df[['driver_id', 'hex_id']] = driver_df[['driver_id', 'hex_id']].astype(int) # set data type to int

        return driver_df

    def gen_riders(self, number_of_riders: int, hr_time: int, resolution: int = Global_Resolution):
        """Sample locations for riders
        
        Sample multiple locations for riders based on the given locational distribution 
        of riders. The default distribution model is KDE.

        Parameters:
            number_of_riders: an int, indicating how many riders are generated.
            hr_time: an int, the value is the hour of the day, indicating which distribution model will be used.
            resolution: an int, determines the number of cells in the map.

        Returns:
            rider_df: a pandas DataFrame, including information of generated riders. 
        """
        rider_locations = self.model.sample_from_model(self.rider_model[f'{hr_time}'], number_of_riders, self.random_seed) # dtype = numpy ndarray
        rider_ids = []
        hex_ids = []
        
        for rider_id, geo_info in enumerate(rider_locations):
            hex_ids.append(self.hexagons_dic[h3.geo_to_h3(geo_info[0], geo_info[1], resolution)])
            rider_ids.append(rider_id)

        x_list, y_list = wgs84_to_xy(rider_locations.T[0], rider_locations.T[1])

        rider_df = pd.DataFrame({'rider_id':rider_ids, 'hex_id':hex_ids, 'x':x_list, 'y':y_list, 'time_step_in_pool':0})
        # rider_df[['rider_id', 'hex_id', 'time_step_in_pool']] = rider_df[['rider_id', 'hex_id', 'time_step_in_pool']].astype(int) # set data type to int

        return rider_df


class RideHailingENV(gym.Env):
    """Simulation environment for project optimising matching radius for ride-hailing system

    This class is the main simulator for the master thesis project optimising matching radius 
    for a ride-hailing system with reinforcement learning. The project is carried out in TU Delft. 
    This simulator is built base on the geographical information of Austin, Texas, USA. It intake
    continous matching radius as the action, and the reward is the total net profit made by the 
    system within a day.

    Attributes:
     lower_bound - lower bound of action space, minimum matching radius, unit is meters.
     upper_bound - upper bound of action space, maximum matching radius, unit is meters.
     hex_h3 - make an instance of HexH3 class, to generate hexagonal cells.
     model - make an instance of Gen_Model class, to generate riders and drivers for the simulator.
     match - make an instance of Match class, to run the matching algorithm.
     hexagons - a list of hexagonal cells in the format of h3 codes, is index of hexagons.
     num_cells - the number of hexagonal cells in the map, the scale of action space.
     radius_initial - initial matching radius when reset the environment, unit is meters.
     driver_num_ini - initial number of drivers, can be changed if set dynamic.
     rider_num_ini - initial number of riders, rider number is changing among different steps.
     fuel_unit_price - average travelling fuel cost per vehicle per kilometer in the US, the unit is US dollars.
     time_window - time interval between every two matching process (Uber Batched matching), fixed among all steps, unit is minutes.
     total_reward - total reward for the intake action.
     gen_rate_rider - overall generating rate of riders, number of riders per time-window.
     gen_rate_driver - active if vehicle number are set dynamic, number of new drivers per time-window.
     ride_price - average ride price in Austin urban area, the value is estimated from Uber ride data in 2022.
     rider_patience - the maximum number of steps a rider can stay in the matching pool.
     p_unmatch_rider - penalty per unmatched rider, the value is cauculated base on the probability of losing a potential ride.
     action_space - defines the numerical range of intake actions.
     observation_space - defines the numerical range of overall observations.
     sub_observation_space - defines the numerical range of observations within a cell.
    """
    def __init__(self, random_seed: int = 1) -> None:

        lower_bound = 50
        upper_bound = 5000
        self.hex_h3 = HexH3()
        self.model = Gen_Model(random_seed=random_seed)
        self.match = Match()
        self.random_seed = random_seed

        self.hexagons = self.hex_h3.get_hexagons(resolution=Global_Resolution, display_map=False)
        self.hex_num = np.size(self.hexagons)
        self.num_cells = np.size(self.hexagons)
        self.radius_initial = 500
        self.driver_num_ini = 100
        self.rider_num_ini = 30
        self.fuel_unit_price = 0.125 * 0.001 # per veh per kilomter -> per meter
        self.time_window = 2
        self.total_reward = 0
        self.gen_rate_rider = 5
        self.gen_rate_driver = 10
        self.ride_price = 23.92
        self.rider_patience = 5
        self.p_unmatch_rider = self.ride_price / self.rider_patience

        self.drivers = None
        self.riders = None
        self.drivers_tmp = None
        self.riders_tmp = None

        self.action_space = spaces.Box(
                low=np.array(lower_bound*np.ones(self.num_cells)), 
                high=np.array(upper_bound*np.ones(self.num_cells)),
                dtype=np.int32
                )
        self.observation_space = spaces.Discrete(self.num_cells*2)
        self.sub_observation_space = spaces.Discrete(7)
        
    def reset(self, time_ini: int = 1) -> np.array:
        """
        reset the environment for the first step in every episode.

        Parameters:
         time_ini -  set the initial time to 0-1 hour of a day.

        Returns:
         returns the initial matching radius.
        """
        hexagons = self.hexagons
        radius = np.ones_like(hexagons, dtype=int) * self.radius_initial
        self.drivers = self.model.gen_drivers(self.driver_num_ini, time_ini)
        self.riders = self.model.gen_riders(self.rider_num_ini, time_ini)

        self.drivers['driver_id'] = np.arange(self.drivers.shape[0])
        self.riders['rider_id'] = np.arange(self.riders.shape[0])

        print("Simulator Initialized!")
        return radius
    
    def step(self, radius: np.array, hr_time: int, rend_step: bool = False) -> tuple[float, dict, list]:
        """
        The main process of a step.

        Parameters:
         radius - matching radius for each cell.
         hr_time - the hourly time step in a day, determines location distribution of riders and drivers.
         rend_step - visualize one step if set to True.

        Returns:
         the reward of one step and matched pairs within this step. 
        """

        self.riders_tmp = self.riders.copy()
        self.drivers_tmp = self.drivers.copy()

        rider_vec = self.riders[['x', 'y']].values
        driver_vec = self.drivers[['x', 'y']].values

        # get the distance matrix and matching pool
        dis_matrix = self.__vector_dis(rider_vec, driver_vec)
        hex_ids = self.riders['hex_id']
        r_radius = radius[hex_ids]

        # get the matching pool
        pool = self.__get_pool(dis_matrix, r_radius)

        # matching process
        match_statue = self.__match(pool)
        reward = self.__execute_match(match_statue)
        self.riders, self.drivers = self.__state_transit(hr_time)

        if rend_step:
            state = [self.riders_tmp, self.drivers_tmp]
            self.render(state, radius, match_statue)
        
        return reward, match_statue
    
    def get_observe(self): # observation is number of riders/drivers in each cell
        rider_counts = self.riders['hex_id'].value_counts().sort_index()
        driver_counts = self.drivers['hex_id'].value_counts().sort_index()
        rider_counts_list = rider_counts.reindex(range(self.hex_num), fill_value=0).tolist()
        driver_counts_list = driver_counts.reindex(range(self.hex_num), fill_value=0).tolist()
        return rider_counts_list, driver_counts_list
    
    def __get_pool(self, dis_matrix: list, radius_set: int) -> np.ndarray:
        """
        form a matching pool for all the riders and available drivers whithin the matching radius.

        Parameters:
            riders - locations, numbers of all the riders.
            drivers - locations, numbers of all the drivers.
            radius - matching radius for each cell, riders in the same cell have the same matching radius.

        Returns:
            returns a list consist all the possible matches and the distance between them.
        """
        match_pool = []
        for i in range(dis_matrix.shape[0]):
            sub_pool = []
            radius_rider = radius_set[i]
            driver = list(np.where(dis_matrix[i] <= radius_rider)[1])
            rider = list(np.ones(np.size(driver), dtype=int)*i)
            dis = list(np.array(dis_matrix[i][dis_matrix[i] <= radius_rider])[0])
            sub_pool.extend([rider])
            sub_pool.extend([driver])
            sub_pool.extend([dis])
            sub_pool = list(map(list, zip(*sub_pool)))
            match_pool.extend(sub_pool)
        return match_pool
            
    def __match(self, pool: list): # MM: Maximum Matching, OM: Optimised Matching
        """
        excute matching algorithm to find the optimal match for the given matching pool.

        Parameters:
         pool - matching pool with distance. 

        Returns:
         match statue with matched pairs and their distance. 
        """
        matched_pairs = self.match.match(pool, method='Munkres')
        return matched_pairs
    
    def __execute_match(self, match_statue:list) -> tuple[float, tuple]:
        """
        apply the matched pairs to the map, update riders and drivers, observe reward and penalty.

        Parameters:
         riders - locations, numbers of all the riders.
         drivers - locations, numbers of all the drivers.
         match_statue - matched pair of riders and drivers with the distance between them.

        Returns:
         reward - the net monetary profit made from the ride-hailing system within a step.
         pool_next - next state of the environment after taking the action.
        """
        # initialise some variables
        reward = 0
        dis = 0

        # iterate over matched pairs
        for rider, driver, distance in match_statue:
            self.riders = self.riders.drop(rider)
            self.drivers = self.drivers.drop(driver)
            dis += distance
            reward += self.ride_price

        # calculate punishment and travel cost, finish the reward function
        reward -= dis * self.fuel_unit_price + self.p_unmatch_rider * len(self.riders) + self.fuel_unit_price * len(self.drivers)

        return reward
    
    def __vector_dis(self, rider_vec, driver_vec):
        m = np.shape(rider_vec)[0]
        n = np.shape(driver_vec)[0]
        M = np.dot(rider_vec, driver_vec.T)
        H = np.tile(np.matrix(np.square(rider_vec).sum(axis=1)).T,(1,n))
        K = np.tile(np.matrix(np.square(driver_vec).sum(axis=1)),(m,1))
        return np.sqrt(-2 * M + H + K)
    
    def __state_transit(self, hr_time: int) -> dict: 
        """
        update the current state and give the state of the next step.

        Parameters:
         state - the current state, locations of riders and drivers.
         hr_time - hourly time of a day, this is used to generate new riders and drivers.

        Returns:
         returns the locations of riders and drivers for the next step.
        """

        # update riders
        rider_size = self.gen_rate_rider * self.time_window
        new_riders = self.model.gen_riders(rider_size, hr_time)
        rider_next = pd.concat((self.riders, new_riders), axis=0)
        rider_next['time_step_in_pool'] += 1
        rider_next = rider_next.drop(rider_next[rider_next['time_step_in_pool']>self.rider_patience].index) # inpatient riders quit the matching pool

        # update drivers
        driver_size = self.driver_num_ini
        driver_next = self.model.gen_drivers(driver_size, hr_time)

        rider_next = rider_next.reset_index(drop=True)
        driver_next = driver_next.reset_index(drop=True)

        # re-index drivers and riders
        driver_index = driver_next.shape[0]
        driver_next['driver_id'] = np.arange(driver_index)
        rider_index = rider_next.shape[0]
        rider_next['rider_id'] = np.arange(rider_index)

        return rider_next, driver_next

    def render(self, state: tuple, radius_set: dict, match_statue: list, color_set: tuple = ['red', 'blue'], folium_map=None) -> None:
        """
        visualise the state and action for one step, red circle is matching range (within matching radius),
        green lines are the links for matched pairs.

        Parameters:
         state - the current state, locations of riders and drivers.
         radius_set - matching radius for each cell.
         match_statue - matched pair of riders and drivers with the distance between them.
         color_set - the color for matching radius and hexagonal cells respectively.
         folium_map - map object.
        """

        cells = self.hexagons
        riders = state[0]
        drivers = state[1]

        matched_riders = pd.DataFrame(match_statue)[0].to_list()
        matched_drivers = pd.DataFrame(match_statue)[1].to_list()
        polylines = []
        lat = []
        lng = []
        matched_rider_location = {}
        matched_driver_location = {}

        for cell in cells:
            cell = str(cell)
            polygons = h3.h3_set_to_multi_polygon([cell], geo_json=False)
            outlines = [loop for polygon in polygons for loop in polygon]
            polyline = [outline + [outline[0]] for outline in outlines][0]
            lat.extend(map(lambda v:v[0],polyline))
            lng.extend(map(lambda v:v[1],polyline))
            polylines.append(polyline)
        if folium_map is None:
            m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=12, tiles='openstreetmap') # titles can also be: 'cartodbpositron'
        else:
            m = folium_map
        for polyline in polylines:
            my_PolyLine=folium.PolyLine(locations=polyline,weight=2,color='blue')
            m.add_child(my_PolyLine)
       
        # add driver markers
        for i in range(drivers.shape[0]):
            driver_wgs = xy_to_wgs84([drivers.loc[i]['x'], drivers.loc[i]['y']])
            folium.Marker(
                location=driver_wgs,
                icon=folium.Icon(
                    color=color_set[0],
                    prefix='fa',
                    icon='car'
                    )
                ).add_to(m)
            if drivers.loc[i]['driver_id'] in matched_drivers:
                matched_driver_location[f'{int(drivers.loc[i]["driver_id"])}'] = driver_wgs
          
        # add rider markers and matching radius
        for j in range(riders.shape[0]):
            rider_wgs = xy_to_wgs84([riders.loc[j]['x'], riders.loc[j]['y']])
            folium.Marker(
                location=rider_wgs,
                icon=folium.Icon(
                    color=color_set[1],
                    prefix='fa',
                    icon='male'
                    )
                ).add_to(m)
            
            folium.Circle(
                    radius=float(radius_set[int(riders.loc[j]['hex_id'])]),
                    location=rider_wgs,
                    color="red",
                    weight=1,
                    fill=True,
                    fill_opacity=0.1
                ).add_to(m)
            if riders.loc[j]['rider_id'] in matched_riders:
                matched_rider_location[f'{int(riders.loc[j]["rider_id"])}'] = rider_wgs
       
        for rider, driver, dis in match_statue:
            folium.PolyLine(
                locations=[matched_rider_location[f'{int(rider)}'], matched_driver_location[f'{int(driver)}']],
                color='green', 
                weight=5,
                tooltip='matched_links'
                ).add_to(m)
    
        display(m)
        pass

In [29]:
env = RideHailingENV(random_seed=1)
env.reset()

Simulator Initialized!


array([500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
       500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
       500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
       500, 500, 500, 500, 500, 500, 500, 500, 500])

In [30]:
model = Gen_Model()
d = model.gen_drivers(100, 1)
d

,driver_id,hex_id,x,y
0,0,12,18299.000053,26579.354418
1,1,26,16743.187196,19227.040267
2,2,27,17478.524612,20329.953486
3,3,38,3019.685036,9585.007035
4,4,21,20092.202976,25441.125854
...,...,...,...,...
95,95,27,15218.351255,20600.317765
96,96,21,19493.978535,22923.082396
97,97,27,16200.322083,20777.177236
98,98,27,16382.388597,21439.591639


In [ ]:
hex = HexH3()
h = hex.get_hexagons()
print(np.size(h))

In [ ]:
# 创建示例 DataFrame
data = {'A': np.random.randint(0, 49, size=100),
        'B': np.random.randint(0, 49, size=100)}
df = pd.DataFrame(data)

# 统计 'A' 列元素出现的次数
counts = df['A'].value_counts().sort_index()

# 使用 reindex 方法构建列表
result_list = counts.reindex(range(49), fill_value=0).tolist()

print(data['A'],result_list)

In [ ]:
env = RideHailingENV()

In [ ]:
env.observation_space.n

In [ ]:
env.action_space.shape[0]

In [ ]:
import datetime
import folium

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from folium.plugins import HeatMap
from sklearn.neighbors import KernelDensity


class Dataset_Info:
    def show_heatmap(coord_data, keyword='start_location', zoom=9, data_format='dataframe'):
        m = folium.Map(location=(30.2672, -97.7431), zoom_start=zoom)
        if data_format == 'dataframe':
            data = list(coord_data.groupby([f'{keyword}_lat', f'{keyword}_long']).groups.keys())
        if data_format == 'ndarray':
            data = coord_data
        heatmap = HeatMap(data, radius=14)
        heatmap.add_to(m)

        return m

    def draw_hist(data, title):
        plt.figure(figsize=(15,2))
        plt.hist(data, density=True, bins=500) 
        plt.ylabel('Probability')
        plt.xlabel('Data')
        plt.title(title)
        plt.grid()
        plt.show()

        pass

    def draw_scatter(data, keyword, title, sample_plot=True, sample_scale=1000, get_sample=False):
        fig = plt.figure(figsize=(10,7))
        if sample_plot == True:
            sample_idx = np.random.choice(data.shape[0], sample_scale, replace=False)
            sampled_reverse = data.loc[sample_idx]
            sampled_data = sampled_reverse[[f'{keyword}_lat', f'{keyword}_long']]
            plt.scatter(sampled_data[f'{keyword}_lat'], sampled_data[f'{keyword}_long'], alpha=0.7)
        else:
            plt.scatter(data[f'{keyword}_lat'], data[f'{keyword}_long'], alpha=0.7)
        plt.ylabel('Probability')
        plt.xlabel('Data')
        plt.title(title)
        plt.grid() 
        plt.show()

        if get_sample == True:
            return sampled_data


class Build_Model:
    def __init__(self) -> None:
        self.rider_dataset_hr = pd.read_csv('./dataset/rider_data_hr.csv')
        self.driver_dataset_hr = pd.read_csv('./dataset/driver_data_hr.csv')
        self.lat_range = [30.16, 30.49] # location gennerating range
        self.lon_range = [-97.90, -97.61] # location gennerating range

        pass
    
    def data_slice(self):
        start_loc = self.dataset[["RIDE_ID", "started_on", "start_location_lat", "start_location_long"]]
        end_loc = self.dataset[["RIDE_ID", "completed_on", "end_location_lat", "end_location_long"]]

        return start_loc, end_loc
    
    def data_filter(self):
        data = self.dataset
        lat_range = self.lat_range # location gennerating range
        lon_range = self.lon_range # location gennerating range
        data = data.drop(data[(data['start_location_lat']<lat_range[0]) | (data['start_location_lat']>lat_range[1])].index)
        data = data.drop(data[(data['start_location_long']<lon_range[0]) | (data['start_location_long']>lon_range[1])].index)
        data = data.drop(data[(data['end_location_lat']<lat_range[0]) | (data['end_location_lat']>lat_range[1])].index)
        data = data.drop(data[(data['end_location_long']<lon_range[0]) | (data['end_location_long']>lon_range[1])].index)
        start_loc, end_loc = self.data_slice()
        rider_dataset_hr = self.add_step_info(start_loc, keyword='started_on', step_size=60)
        driver_dataset_hr = self.add_step_info(end_loc, keyword='completed_on', step_size=60)
        rider_dataset_hr.to_csv('../dataset/rider_data_hr.csv')
        driver_dataset_hr.to_csv('../dataset/driver_data_hr.csv')

        pass

    def add_step_info(self, data, keyword, step_size=60):
        # keyword: which column contains time data, step_size: how many minuetes per time step
        step_dataset = data.copy()
        step_dataset[f'{keyword}'] = pd.to_datetime(step_dataset[f'{keyword}'])
        step_dataset[f'{keyword}'] = step_dataset[f'{keyword}'].dt.time
        step_dataset.insert(step_dataset.shape[1], 'time_step_index', 1)
        number_step = int(1440 / step_size)

        start_time = datetime.datetime.strptime('00:00:00', '%H:%M:%S')
        time_debug = datetime.datetime.strptime('1900-01-02 00:00:00', '%Y-%m-%d %H:%M:%S') - datetime.timedelta(minutes=step_size)

        for i in range(number_step):
            if start_time == time_debug:
                end_time = start_time + datetime.timedelta(minutes=step_size) - datetime.timedelta(seconds=1)
            else:
                end_time = start_time + datetime.timedelta(minutes=step_size)
            start_time_f = start_time.time()
            end_time_f = end_time.time()
            step_index = i + 1
            step_dataset.loc[(step_dataset[f'{keyword}'] > start_time_f) & (step_dataset[f'{keyword}'] <= end_time_f), 'time_step_index'] = step_index
            start_time = datetime.datetime.strptime(str(end_time_f), '%H:%M:%S')

        return step_dataset

    def sample_from_model(self, model: object, num_sample: int = 1000, seed = 1) -> list:
        sample = model.sample(num_sample, random_state=seed)
        #sample_df = pd.DataFrame(sample, columns=['sampled_location_lat', 'sampled_location_long'])

        return sample

    def fit_model(self, data: pd.DataFrame, keyword: str, band_width:float = 0.0035) -> dict:
        # keyword: 'driver' or 'rider'
        model_set = {}

        if keyword == 'driver':
            index_key = 'end_location'
        else:
            index_key = 'start_location'

        for time in range(1,25):
            data_hr = data[data['time_step_index']==time]
            xy_train  = np.vstack([data_hr[f'{index_key}_lat'], data_hr[f'{index_key}_long']]).T
            kde_skl = KernelDensity(kernel='gaussian', bandwidth=band_width)
            model_set[f'{time}'] = kde_skl.fit(xy_train)

        return model_set
    
    def get_model(self):
        rider_model = self.fit_model(self.rider_dataset_hr, keyword='rider')
        driver_model = self.fit_model(self.driver_dataset_hr, keyword='driver')
        
        return rider_model, driver_model

In [ ]:
model = Build_Model()

In [ ]:
model_1, model_2 = model.get_model()
sample = model.sample_from_model(model_1['1'], 10)
sample